In [2]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), 'resnet'))
sys.path.append(parent_dir)

import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchmetrics import Accuracy

import torch.optim as optim
import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import copy
import gc
import math

import warnings
warnings.filterwarnings('ignore')

from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

from resnet_18 import *

from scipy.integrate import simpson

In [2]:
print(torch.cuda.is_available())

True


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
%run utils.ipynb

In [5]:
batch_size = 8
train_path = '../datasets/imagenette2/train'
val_path = '../datasets/imagenette2/val'

In [6]:
train_dataloader = DataLoader(datasets.ImageFolder(train_path, transform = transforms.Compose([
                                                                    transforms.RandomResizedCrop(224),
                                                                    transforms.RandomHorizontalFlip(),
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std=[0.229, 0.224, 0.225])
                                                            ])), batch_size = batch_size, shuffle=True, num_workers=4, pin_memory=True)

test_dataloader = DataLoader(datasets.ImageFolder(val_path,
                                                               transform=transforms.Compose([
                                                                   transforms.ToTensor(),
                                                                   transforms.Resize([224, 224]),
                                                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                        std=[0.229, 0.224, 0.225])
                                                               ])),batch_size=batch_size, shuffle=True,num_workers=4, pin_memory=True)


In [7]:
classes = ['tench', 'springer', 'casette_player', 'chain_saw','church', 'French_horn', 'garbage_truck', 'gas_pump', 'golf_ball', 'parachute']


In [8]:
criterion = nn.CrossEntropyLoss(reduction="mean").cuda()

## Sparsity

In [9]:
%run metrics.ipynb

In [ ]:
MODEL_PATH = "models/0_model_lt.pth.tar"
model = torch.load(MODEL_PATH, map_location=device, weights_only=False)
model.to(device)

sparse_saliency = filter_and_compute_sparsity(model, test_dataloader,"Saliency", device, resnet=True)
print("The normal vanilla sparsity score is: ", sparse_saliency)

sparse_ig =  filter_and_compute_sparsity(model, test_dataloader,"IntegratedGradients", device, resnet=True)
print("The normal ig sparsity score is: ",sparse_ig)

del model

In [ ]:
MODEL_PATH = "models/1_10_model_lt.pth.tar"
model = torch.load(MODEL_PATH, map_location=device, weights_only=False)
model.to(device)

sparse_saliency = filter_and_compute_sparsity(model, test_dataloader,"Saliency", device, resnet=True)
print("The normal vanilla sparsity score is: ", sparse_saliency)

sparse_ig =  filter_and_compute_sparsity(model, test_dataloader,"IntegratedGradients", device, resnet=True)
print("The normal ig sparsity score is: ",sparse_ig)

del model

In [ ]:
MODEL_PATH = "models/1_model_lt.pth.tar"
model = torch.load(MODEL_PATH, map_location=device, weights_only=False)
model.to(device)

sparse_saliency = filter_and_compute_sparsity(model, test_dataloader,"Saliency", device, resnet=True)
print("The normal vanilla sparsity score is: ", sparse_saliency)

sparse_ig =  filter_and_compute_sparsity(model, test_dataloader,"IntegratedGradients", device, resnet=True)
print("The normal ig sparsity score is: ",sparse_ig)

del model

In [ ]:
MODEL_PATH = "models/2_model_lt.pth.tar"
model = torch.load(MODEL_PATH, map_location=device, weights_only=False)
model.to(device)

sparse_saliency = filter_and_compute_sparsity(model, test_dataloader,"Saliency", device, resnet=True)
print("The normal vanilla sparsity score is: ", sparse_saliency)

sparse_ig =  filter_and_compute_sparsity(model, test_dataloader,"IntegratedGradients", device, resnet=True)
print("The normal ig sparsity score is: ",sparse_ig)

del model

In [ ]:
MODEL_PATH = "models/12_model_lt.pth.tar"
model = torch.load(MODEL_PATH, map_location=device, weights_only=False)
model.to(device)

sparse_saliency = filter_and_compute_sparsity(model, test_dataloader,"Saliency", device, resnet=True)
print("The normal vanilla sparsity score is: ", sparse_saliency)

sparse_ig =  filter_and_compute_sparsity(model, test_dataloader,"IntegratedGradients", device, resnet=True)
print("The normal ig sparsity score is: ",sparse_ig)

del model

In [ ]:
MODEL_PATH = "models/29_model_lt.pth.tar"
model = torch.load(MODEL_PATH, map_location=device, weights_only=False)
model.to(device)

sparse_saliency = filter_and_compute_sparsity(model, test_dataloader,"Saliency", device, resnet=True)
print("The normal vanilla sparsity score is: ", sparse_saliency)

sparse_ig =  filter_and_compute_sparsity(model, test_dataloader,"IntegratedGradients", device, resnet=True)
print("The normal ig sparsity score is: ",sparse_ig)

del model

In [10]:
normal_sparse_saliency = 0.4225200890204471
normal_sparse_ig = 0.5886364333326195

prune_10_sparse_saliency = 0.46837122128811204
prune_10_sparse_ig = 0.6066886995654848

prune_20_sparse_saliency = 0.45473032477062864
prune_20_sparse_ig = 0.6112264423854529

prune_30_sparse_saliency = 0.4541549643816636
prune_30_sparse_ig = 0.6093546384460925

prune_50_sparse_saliency =0.44388624278818267
prune_50_sparse_ig = 0.5999048563523421

prune_70_sparse_saliency = 0.4480548349520132
prune_70_sparse_ig = 0.595999840949814

In [11]:
vanilla_gradient = [normal_sparse_saliency, prune_10_sparse_saliency, prune_20_sparse_saliency, prune_30_sparse_saliency, prune_50_sparse_saliency,prune_70_sparse_saliency ]
integrated_gradients = [normal_sparse_ig, prune_10_sparse_ig, prune_20_sparse_ig, prune_30_sparse_ig, prune_50_sparse_ig,prune_70_sparse_ig ]

row_labels = ["Normal Model", "Pruned 10%", "Pruned 20%", "Pruned 30%","Pruned 50%", "Pruned 70%"]

data = {
    "Vanilla Gradient": vanilla_gradient,
    "Integrated Gradients": integrated_gradients
}

df = pd.DataFrame(data, index = row_labels)

df.style.set_table_styles(
    [{'selector': 'th, td', 'props': [('border', '1px solid black')]}])

,Vanilla Gradient,Integrated Gradients
Normal Model,0.422520,0.588636
Pruned 10%,0.468371,0.606689
Pruned 20%,0.454730,0.611226
Pruned 30%,0.454155,0.609355
Pruned 50%,0.443886,0.599905
Pruned 70%,0.448055,0.596000
